In [5]:
# https://ipython.org/ipython-doc/3/config/extensions/``autoreload.html
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
import os, sys
ipynb_path = os.path.dirname(os.path.realpath("__file__"))
print(ipynb_path)

import sys
sys.path.append(os.path.join(ipynb_path, ".."))

C:\Users\harry\Documents\code_ws\pb_ws\debug_pyplanners\itj


In [9]:
from termcolor import cprint
from load_path import *
from run import get_itj_pddl_problem_from_json

In [54]:
debug_problem_name = "nine_pieces_process_symbolic.json"
input_problem, _, _ = get_itj_pddl_problem_from_json(debug_problem_name, use_partial_order=True, debug=True, 
                                                    reset_to_home=0,
                                                    use_fluents=1)

Symbolic process json parsed from C:\Users\harry\Documents\code_ws\pb_ws\debug_pyplanners\itj\nine_pieces_process_symbolic.json
Using beam sequence ordering: ['b0', 'b1', 'b2', 'b3', 'b5', 'b6', 'b7', 'b8', 'b4']


In [21]:
from pddlstream.algorithms.algorithm import parse_problem
from pddlstream.algorithms.constraints import PlanConstraints
from pddlstream.utils import INF, Verbose, str_from_object, elapsed_time

constraints = PlanConstraints()
unit_costs = True
max_iterations=INF
max_time=INF
max_memory=INF

In [22]:
from pddlstream.algorithms.incremental import solve_incremental
from pddlstream.algorithms.incremental import solve_incremental
from pddlstream.language.constants import print_plan, is_plan
from pddlstream.algorithms.meta import solve

# solution = solve_incremental(input_problem, constraints=constraints,
#                   unit_costs=1, success_cost=INF,
#                   max_iterations=INF, max_time=INF, max_memory=INF,
#                   initial_complexity=0, complexity_step=1, max_complexity=INF,
#                   verbose=False)
solution = solve(input_problem, algorithm='incremental',
                 max_time=60,
                 unit_costs=True,
                 max_planner_time=300,
                 debug=0, verbose=0) #, planner=discrete_planner)

plan, cost, evaluations = solution
plan_success = is_plan(plan)

print('-'*10)
print_plan(plan)
cprint('Planning {}'.format('succeeds' if plan_success else 'fails'), 'green' if plan_success else 'red')

Iteration: 1 | Complexity: 0 | Calls: 0 | Evaluations: 161 | Solved: False | Cost: inf | Search Time: 0.001 | Sample Time: 0.000 | Time: 0.001
Initial: State{Atom atrack(v1), Atom atrack(v10), Atom atrack(v11), Atom atrack(v12), Atom atrack(v13), Atom atrack(v14), Atom atrack(v15), Atom atrack(v16), Atom atrack(v2), Atom atrack(v3), Atom atrack(v4), Atom atrack(v5), Atom atrack(v6), Atom atrack(v7), Atom atrack(v8), Atom atrack(v9), Atom notoolatjoint(v1, v3), Atom notoolatjoint(v1, v4), Atom notoolatjoint(v1, v6), Atom notoolatjoint(v2, v3), Atom notoolatjoint(v2, v4), Atom notoolatjoint(v2, v8), Atom notoolatjoint(v3, v1), Atom notoolatjoint(v3, v2), Atom notoolatjoint(v4, v1), Atom notoolatjoint(v4, v2), Atom notoolatjoint(v5, v6), Atom notoolatjoint(v5, v9), Atom notoolatjoint(v6, v1), Atom notoolatjoint(v6, v5), Atom notoolatjoint(v7, v8), Atom notoolatjoint(v7, v9), Atom notoolatjoint(v8, v2), Atom notoolatjoint(v8, v7), Atom notoolatjoint(v9, v5), Atom notoolatjoint(v9, v7), Ato

In [86]:
ac = solution.plan[0]
type(ac)

pddlstream.language.constants.Action

In [66]:
from pddlstream.algorithms.downward import get_problem, task_from_domain_problem
from pddlstream.algorithms.algorithm import parse_problem
from pddlstream.algorithms.instantiate_task import sas_from_pddl, instantiate_task, convert_instantiated
from pddlstream.algorithms.incremental import process_stream_queue
from pddlstream.language.attachments import compile_fluents_as_attachments
from pddlstream.algorithms.instantiation import Instantiator
from pddlstream.algorithms.common import SolutionStore
from pddlstream.utils import INF, Verbose, str_from_object, elapsed_time

evaluations, goal_expression, domain, externals = parse_problem(
    input_problem, constraints=constraints, unit_costs=unit_costs)

static_externals = compile_fluents_as_attachments(domain, externals)
instantiator = Instantiator(static_externals, evaluations)

store = SolutionStore(evaluations, INF, INF, 0, max_memory=INF) # TODO: include other info here?
process_stream_queue(instantiator, store, 0, verbose=0)

########################

problem = get_problem(evaluations, goal_expression, domain, unit_costs)
task = task_from_domain_problem(domain, problem)

with Verbose(0):
    instantiated = instantiate_task(task)
    instantiated = convert_instantiated(instantiated)

In [87]:
instantiated.actions[0].action.name

'pick_element_from_rack'

In [111]:
# 'b0' in 
[obj_from_pddl(v).value for v in instantiated.actions[0].var_mapping.values()]

['b0', 'g2']

In [115]:
obj_from_pddl('v11').value

'c2'

In [169]:
from pddlstream.language.conversion import pddl_from_object, obj_from_pddl

def add_applied_effects(action):
    action.applied_effects = []
    for effects in [action.add_effects, action.del_effects]:
        negate = (effects is action.del_effects)
        for i, (conditions, effect) in reversed(list(enumerate(effects))):
            if all(literal_holds(state, literal) or (literal in axiom_from_atom) for literal in conditions):
                action.precondition.extend(conditions)
                effects[i] = ([], effect)
                action.applied_effects.append(effect.negate() if negate else effect)
            else:
                effects.pop(i)
    return action

applied_iac = []
for iac in instantiated.actions:
    if iac.action.name == ac.name and all(arg in [obj_from_pddl(v).value for v in iac.var_mapping.values()] for arg in ac.args):
        # print('Found')
        # iac.dump()
        # print(ac)
        # print('===')
        iac = add_applied_effects(iac)
        applied_iac.append(iac)
        break

In [170]:
applied_iac[0].applied_effects

[<Atom canfreemove()>,
 <Atom robotgripperempty()>,
 <Atom attached(v15)>,
 <NegatedAtom atrack(v15)>,
 <NegatedAtom robottoolchangerempty()>]

In [163]:
initial_state = set(instantiated.atoms)

In [164]:
set(eff[1] for eff in applied_iac[0].del_effects)

{<Atom atrack(v15)>, <Atom robottoolchangerempty()>}

In [165]:
state = initial_state | set(eff[1] for eff in applied_iac[0].add_effects)

In [166]:
state -= set(eff[1] for eff in applied_iac[0].del_effects)

In [172]:
import pddl
from pddlstream.algorithms.scheduling.reinstantiate import reinstantiate_action, reinstantiate_action_instances 

reinstantiate_action_instances(task, applied_iac)
# new_instance = reinstantiate_action(init_state, ac)
# pddl_ac = pddl.PropositionalAction(instance.name, precondition, new_effects, instance.cost, action, var_mapping)

[<PropositionalAction '(pick_tool_from_rack v15)' at 0x1b5da346a88>,
 <PropositionalAction '@goal' at 0x1b5da346b08>]

In [174]:
solution.plan[1]

Action(name='pick_element_from_rack', args=('b0', 'g2'))

In [175]:
instance = instantiator.pop_stream()

IndexError: index out of range

## Debug types

In [50]:
from pddlstream.language.conversion import is_atom, is_negated_atom, objects_from_evaluations, pddl_from_object, \
    pddl_list_from_expression, obj_from_pddl

# pddl_from_object()
obj_from_pddl('v1')

b0